# Pair Programming 
## Lección 13 - Limpieza V - Valores Nulos Sklearn
​
En este caso trabajaremos con el dataframe que limpiamos en el ejercicio de pair de Limpieza III donde limpiabamos las columnas de species, age, etc. y eliminabamos los outliers. Hoy volveremos a gestionar valores nulos, pero en este caso usaremos los métodos de imputación de sklearn.

Antes de seguir, recordamos las preguntas que nos planteamos al principio del pair programming de EDA para dirigir nuestro análisis.

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. **Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

variable    ¿Está limpia?

age         ✔️ esta en formato string cuando debería ser integer y en algunos casos tenemos rangos de edad

species     ✔️ es un jaleo! Debemos unificar los nombres y reducir a las especies más importantes

country     ✔️ los paises están en mayúsculas, algunos se repiten con algunos cambios

fatal       ✔️ la limpiamos en el pair de Pandas V

year        ✔️ es una columna de tipo float deberíamos convertirla a integer

sex         ✔️ la limpiamos en el pair de Pandas V

fecha       ✔️ la limpiamos en el pair de Pandas V

Es el momento de ponernos a trabajar con los valores nulos de nuevo 💪🏽. A lo largo de este ejercicio de pair programming vamos a intentar eliminar los valores nulos de nuestras columnas. En la lección hemos aprendido varios métodos de skelarn intentemos aplicarlos todos. Manos a la obra!

In [1]:
import pandas as pd
import sidetable
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [2]:
df = pd.read_csv("limpieza3.csv", index_col = 0)

In [3]:
df.reset_index(inplace = True)

In [4]:
df.head(3)

,index,type,country,activity,age,species_,mes,fatal,sex,especie_tiburon,year,age_ok
0,0,Boating,usa,Paddling,57,White shark,Jun,N,F,White shark,2018,57.0
1,1,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M,Tiger shark,2018,18.0
2,2,Unprovoked,usa,Walking,15,"Bull shark, 6'",May,N,M,Bull shark,2018,15.0


1. Es el momento de eliminar los nulos:
- Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [5]:
nulos = df.stb.missing()
nulos

,missing,total,percent
mes,214,1667,12.837433
age_ok,154,1667,9.238152
age,150,1667,8.998200
species_,123,1667,7.378524
especie_tiburon,123,1667,7.378524
fatal,97,1667,5.818836
activity,31,1667,1.859628
sex,15,1667,0.899820
country,10,1667,0.599880
index,0,1667,0.000000


In [6]:
def aplicar_simple_imputer(dataframe, columna, estrategia):
    # dataframe = dataframe original
    # columna = nombre de la columna a sustituir los valores nulos
    # estrategia = estadístico a aplicar a los valores nulos

    # Iniciar el método, especificar el valor a sustituir los nulos, copy=False para sustituir en la misma columna
    imputer = SimpleImputer(strategy = estrategia, missing_values = np.nan, copy = False)
    # Para ajustar el imputer en la columna a sustituir los nulos
    imputer = imputer.fit(dataframe[[columna]])
    # Para rellenar los nulos con la estrategia que se especifico en el inicializador de la clase SimpleImputer
    dataframe[columna] = imputer.transform(dataframe[[columna]])
    return dataframe

In [7]:
df_simple_age = df.copy()
df_simple_age = aplicar_simple_imputer(df_simple_age, "age_ok", "mean")

In [8]:
nulos = df_simple_age.stb.missing()
nulos

,missing,total,percent
mes,214,1667,12.837433
age,150,1667,8.998200
species_,123,1667,7.378524
especie_tiburon,123,1667,7.378524
fatal,97,1667,5.818836
activity,31,1667,1.859628
sex,15,1667,0.899820
country,10,1667,0.599880
index,0,1667,0.000000
type,0,1667,0.000000


- Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.

💡 Pista 💡 La moda en este tipo de aproximación se indica como most_frequent.

In [9]:
df = aplicar_simple_imputer(df, "sex", "most_frequent")

In [10]:
nulos = df.stb.missing()
nulos

,missing,total,percent
mes,214,1667,12.837433
age_ok,154,1667,9.238152
age,150,1667,8.998200
species_,123,1667,7.378524
especie_tiburon,123,1667,7.378524
fatal,97,1667,5.818836
activity,31,1667,1.859628
country,10,1667,0.599880
index,0,1667,0.000000
type,0,1667,0.000000


Sólo teníamos 15 registros nulos

- Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [11]:
nulos = df.stb.missing()
nulos

,missing,total,percent
mes,214,1667,12.837433
age_ok,154,1667,9.238152
age,150,1667,8.998200
species_,123,1667,7.378524
especie_tiburon,123,1667,7.378524
fatal,97,1667,5.818836
activity,31,1667,1.859628
country,10,1667,0.599880
index,0,1667,0.000000
type,0,1667,0.000000


La columna type no tiene valores nulos, por lo que no vamos a aplicar el método.

- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

Sólo tenemos la variable numérica age_ok, por lo que nos creamos una copia del dataframe original y así poder visualizar como se eliminan los valores nulos con el método KNNImputer.

In [12]:
def aplicar_knn_imputer(dataframe, vecino):
    # solo para variables numéricas

    # dataframe = dataframe original
    # vecino = número entero para medir la distancia entre cada punto y las muestras más cercanas

    # Generar un nuevo dataframe solo con las variables numéricas
    dataframe_numerico = dataframe.select_dtypes(include = np.number)

    # Iniciar el método, toma el valor medio de los vecinos no nulos más cercanos al valor que falta.
    imputer = KNNImputer(n_neighbors = vecino)

    # Se aplica a todo el dataframe
    imputer.fit(dataframe_numerico)
    
    # Se reemplazan los valores nulos
    # imputer.transform(dataframe_numerico) --> devuelve un array
    df_numericas_trans = pd.DataFrame(imputer.transform(dataframe_numerico), columns = dataframe_numerico.columns)

    # Lista de los nombres de las columnas sin nulos
    columnas = df_numericas_trans.columns

    # Vamos a sustituir las columnas numéricas del dataframe por las que hemos generado en numericas_trans sin nulos
    # Primero eliminar las columnas del dataframe original
    dataframe.drop(columnas, axis = 1, inplace = True)
    # Creamos las columnas eliminadas ahora sin nulos
    dataframe[columnas] = df_numericas_trans[columnas]

    return dataframe

In [13]:
df_knn_imputer_age = df.copy()
#df_knn_imputer_age.reset_index(inplace = True)
df_knn_imputer_age = aplicar_knn_imputer(df_knn_imputer_age, 5)

In [14]:
nulos = df_knn_imputer_age.stb.missing()
nulos

,missing,total,percent
mes,214,1667,12.837433
age,150,1667,8.998200
species_,123,1667,7.378524
especie_tiburon,123,1667,7.378524
fatal,97,1667,5.818836
activity,31,1667,1.859628
country,10,1667,0.599880
type,0,1667,0.000000
sex,0,1667,0.000000
index,0,1667,0.000000


- Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [15]:
def aplicar_iterative_imputer(dataframe, estrategia):
    # solo para variables numéricas

    # dataframe = dataframe original
    # estrategia = estadístico a aplicar a los valores nulos en formato string

    # Generar un nuevo dataframe solo con las variables numéricas
    dataframe_numerico = dataframe.select_dtypes(include = np.number)

    # Iniciar el método, especificar las caracteristicas para el valor nulo a sustituir
    # Utilizamos todas las columnas (n_nearest_features) y con menos valores nulos (imputation_order)
    imputer = IterativeImputer(n_nearest_features = None, initial_strategy = estrategia, imputation_order = "ascending")
    # Se aplica a todo el dataframe
    imputer.fit(dataframe_numerico)
    # Se reemplazan los valores nulos
    # imputer.transform(dataframe_numerico) --> devuelve un array
    df_numericas_trans = pd.DataFrame(imputer.transform(dataframe_numerico), columns = dataframe_numerico.columns)

    # Lista de los nombres de las columnas sin nulos
    columnas = df_numericas_trans.columns

    # Vamos a sustituir las columnas numéricas del dataframe por las que hemos generado en numericas_trans sin nulos
    # Primero eliminar las columnas del dataframe original
    dataframe.drop(columnas, axis = 1, inplace = True)
    # Creamos las columnas eliminadas ahora sin nulos
    dataframe[columnas] = df_numericas_trans[columnas]

    return dataframe

In [16]:
df_iterative_age = df.copy()
df_iterative_age = aplicar_iterative_imputer(df_iterative_age, "median")

In [17]:
nulos = df_iterative_age.stb.missing()
nulos

,missing,total,percent
mes,214,1667,12.837433
age,150,1667,8.998200
species_,123,1667,7.378524
especie_tiburon,123,1667,7.378524
fatal,97,1667,5.818836
activity,31,1667,1.859628
country,10,1667,0.599880
type,0,1667,0.000000
sex,0,1667,0.000000
index,0,1667,0.000000


- ¿Podríais explicar qué diferencia hay entre estos dos últimos métodos?

El método Iterative Imputer se basa en las correlacciones de las diferentes columnas con las que trabaja para la estimación de los valores nulos.

El método KNNImputer se basa en la media de sus vecinos más cercanos, es decir toma la media de los vecinos más cercanos con valores no nulos para estimar los valores nulos.

2. Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

Guardamos df_iterative_age porque es el que tiene ya todas las columnas solicitadas sin nulos.

In [18]:
df_iterative_age.to_csv("limpieza_metodos5.csv")

Happy coding 🦈